### Q-8. Quora question pair similarity, you need to find the Similarity between two questions by mapping the words in the questions using TF-IDF, and using a supervised Algorithm you need to find the similarity between the questions. Dataset This is the Dataset You can use this dataset for this question.

In [24]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('max_colwidth', None)

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0


In [5]:
df.shape

(404290, 6)

In [6]:
df = df.sample(100000, random_state=42)

In [7]:
df = df.drop(["id", "qid1", "qid2"], axis=1)

In [8]:
df.isna().sum()

question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.head()

,question1,question2,is_duplicate
8067,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0
368101,What are some of the best side dishes for crab cakes?,What are some good side dishes for buffalo chicken?,0
70497,"Which is more advisable and better material for a crash test in automobiles, ductile or brittle?",What is the best server setup for buddypress?,0
226567,How do I improve logical programming skills?,How can I improve my logical skills for programming?,1
73186,How close we are to see 3rd world war?,How close is a World War III?,1


In [11]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aneesaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
def load_object(file_path):
    try:
        with open(file_path, "rb") as file:
            return pickle.load(file)
    except Exception as err:
        raise CustomException(sys, err)


# Define a function to map the part of speech tags from NLTK to WordNet
def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNet lemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()


def message_text_process(mess):
    mess = remove_word_with_digit_and_non_ascii(mess)
    # remove punctuation from string
    no_punctuation = [char for char in mess if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)  # joining all the characters without punctuation
    # remove stop words
    words = [word.lower() for word in no_punctuation.split() if word not in stopwords.words('english')]
    words = " ".join(words)
    # lemmatize the text
    tokens = nltk.word_tokenize(words)
    lemmas = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, pos=get_wordnet_pos(token))
        lemmas.append(lemma)
    return lemmas


def remove_word_with_digit_and_non_ascii(sentence):
    final_sentence = ""
    sentence = ''.join(c for c in sentence if ord(c) < 128)
    for word in sentence:
        if not any(char.isdigit() for char in word):
            final_sentence += word
    return final_sentence

In [13]:
from sklearn.feature_extraction.text import CountVectorizer  # import Count Vectorizer to create DTM
from sklearn.feature_extraction.text import TfidfTransformer  # import to convert DTM to TF-IDF
from sklearn.pipeline import Pipeline


count_vectorized = CountVectorizer(analyzer=message_text_process)
tfidf_transformer = TfidfTransformer()

text_pipeline = Pipeline(steps=[
    ("bag", count_vectorized),
    ("ttfidf", tfidf_transformer)
])

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = df.drop(["is_duplicate"], axis=1)
y = df["is_duplicate"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
question1_train = text_pipeline.fit_transform(X_train["question1"])
question2_train = text_pipeline.transform(X_train["question2"])

In [18]:
question1_test = text_pipeline.transform(X_test["question1"])
question2_test = text_pipeline.transform(X_test["question2"])

In [19]:
from scipy import sparse

train_arr = sparse.hstack((question1_train, question2_train))
test_arr = sparse.hstack((question1_test, question2_test))

In [95]:
from sklearn import svm

svc = svm.SVC(kernel='poly', coef0=0.5, degree=3, verbose=True).fit(train_arr, y_train) 

[LibSVM]....................................................................................................*........................................*
optimization finished, #iter = 140766
obj = -10677.829722, rho = -0.623198
nSV = 59268, nBSV = 2608
Total nSV = 59268


In [96]:
y_pred = svc.predict(test_arr)

In [97]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.87      0.84     18847
           1       0.75      0.66      0.70     11153

    accuracy                           0.79     30000
   macro avg       0.78      0.76      0.77     30000
weighted avg       0.79      0.79      0.79     30000



In [98]:
from sklearn.naive_bayes import MultinomialNB

class_prior = [0.7, 0.7]

nb = MultinomialNB(alpha=1, class_prior=class_prior).fit(train_arr, y_train)

In [99]:
y_pred = nb.predict(test_arr)

In [100]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.77      0.77     18847
           1       0.62      0.62      0.62     11153

    accuracy                           0.71     30000
   macro avg       0.69      0.69      0.69     30000
weighted avg       0.71      0.71      0.71     30000



In [102]:
import pickle

with open("model.pkl", "wb") as file_obj:
    pickle.dump(svc, file_obj)